<a href="https://colab.research.google.com/github/Queuzy/MaskRCNN_Tutorial/blob/main/MaskRCNN_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mask R-CNN Beginner's Tutorial


## Install Google drive


In [ ]:
# Connect Google drive with authentication
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


# Set root directory of the project
import os, sys, datetime
ROOT_DIR = "/content/gdrive/MyDrive/MaskRCNN_Tutorial"
sys.path.insert(0, ROOT_DIR)
%ls $ROOT_DIR # List file info of root dir


# Set training phase with time stamp
train_phase = "OralCancer_{:%Y%m%dT%H%M}".format(datetime.datetime.now())
print("\n[Info] Set training phase: {}".format(train_phase))

## Import required library

In [ ]:
# Import common Python library
import json
import skimage
import numpy as np

# Import Mask RCNN from custom package
from mrcnn import utils
from mrcnn import visualize
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib

## Configurations

1. Set model configuration by the ModelConfig class.

In [ ]:
class ModelConfig(Config):
    """Configuration for training on the dataset.
     - Derives from the base Config class and overrides some values.
    """
    # Configuration of recognizable namespace
    NAME = "OralCancer"

    # Directory of specific dataset
    DATASET_DIR = ROOT_DIR + "/DATASET/DATA_220624"
    
    # Directory to save training logs and checkpoints
    LOG_DIR = ROOT_DIR + "/LOG/" + train_phase
    
    # Directory to save output files
    OUTPUT_DIR = ROOT_DIR + "/OUTPUT/" + train_phase
    
    # Number of training epochs
    EPOCH = 10
    
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 10

    # Initial weight selection
    # [Option]: coco, imagenet, last, <trained_path>.
    WEIGHT = "coco"
    
    # Batch size: Adjust according to GPU capacity.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + number of classes

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


2. Dataset configuration for loading source images and annotations.

In [ ]:
class OralDataset(utils.Dataset):

    def load_dataset(self, dataset_dir, subset):
        """ Load a subset of the dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load (train, val or test).
        """
        
        # Build class info list
        Source = config.NAME
        self.add_class(Source, 1, "G")
        self.add_class(Source, 2, "Y")
        self.add_class(Source, 3, "R")
        print("\n[INFO] self.class_info: \n{}".format(self.class_info))
        
        # Train, validation or testing dataset?
        assert subset in ["train", "val", "test"], "Please choose from train/val/test set."
        dataset_dir = os.path.join(dataset_dir, subset)
        
        # Load annotations
        # VGG Image Annotator (up to version 2.0.11) saves each image in the form:
        # {
        #   "filename_ex.jpg83251": {
        #     "filename": "filename_ex.jpg",
        #     "size": 83251,
        #     "regions": [
        #       {
        #         "shape_attributes": {
        #           "name": "polygon",
        #           "all_points_x": [...],
        #           "all_points_y": [...]
        #         },
        #         "region_attributes": {
        #           "OralCancer": "R"
        #         }
        #       }
        #     ],
        #     "file_attributes": {}
        #   },
        #   {...}
        # }
        
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, subset+".json")))
        annotations = list(annotations.values())  # Remove the dict keys
        
        # The VIA tool saves images in the JSON even if they don't have any annotations.
        # Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Parse annotation info and append as data queue
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stored in the
            # shape_attributes. (See json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]
            
            # Encode class name by class ID
            label_cls = [r['region_attributes'][Source] for r in a['regions']]
            class_ids = {'G': 1, 'Y': 2, 'R': 3}
            label_ids = [class_ids[a] for a in label_cls]
            
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            # Perhaps you have to consider write size info within preprocessing.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            
            self.add_image(
                Source,
                image_id=a['filename'], # use file name as a unique image id
                class_ids=label_ids,
                path=image_path,
                width=width,height=height,
                polygons=polygons
            )
    
    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a oral dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != config.NAME:
            return super(self.__class__, self).load_mask(image_id)
        
        class_ids = image_info["class_ids"]
        class_ids = np.array(class_ids, dtype=np.int32)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
        
        return mask.astype(np.bool_), class_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == config.NAME:
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

## Develop training progress

In [ ]:
def train(model):
    # Load training dataset
    dataset_train = OralDataset()
    dataset_train.load_dataset(config.DATASET_DIR, "train")
    dataset_train.prepare()

    # Constrain the size of training dataset,
    # since the computational amount of Colab is pathetic lol.
    # image_id = random.choice(dataset_test.image_ids, 9)
    dataset_train = random.choice(dataset_train, 50)
    print('\nSize of train set = {}'.format(len(dataset_train)))

    # Load validation dataset
    dataset_val = OralDataset()
    dataset_val.load_dataset(config.DATASET_DIR, "val")
    dataset_val.prepare()
    print('\nSize of val set = {}'.format(len(dataset_val)))

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long.
    # Also, no need to train all layers, just the heads should do it.
    print("Training network heads")
    
    model.train(dataset_train, dataset_val,
          learning_rate=config.LEARNING_RATE,
          epochs=config.EPOCH,
          layers='heads'
    )

## Train on the dataset

In [ ]:
if __name__ == '__main__':
    
    # Load configuration of parameters
    config = ModelConfig()
    config.display()

    # Build prototype of model
    model = modellib.MaskRCNN(mode="training", config=config, model_dir=config.LOG_DIR)

    # Select initial weights type to load
    if config.WEIGHT.lower() == "coco":
        weights_path = "mask_rcnn_coco.h5" #COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    elif config.WEIGHT.lower() == "imagenet":
        # Start from ImageNet trained weights
        weights_path = model.get_imagenet_weights()
    elif config.WEIGHT.lower() == "last":
        # Find last trained weights
        weights_path = model.find_last()
    else:
        weights_path = config.WEIGHT
    
    # Load initial weights
    print("Loading weights ", weights_path)
    if config.WEIGHT.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes
        model.load_weights(weights_path, by_name=True, exclude=[
                  "mrcnn_class_logits", "mrcnn_bbox_fc",
                  "mrcnn_bbox", "mrcnn_mask"])
    else:
      model.load_weights(weights_path, by_name=True)
    
    # Train model
    train(model)

## Exhibit training curves via **Tensorboard**:

In [ ]:
%load_ext tensorboard
#%reload_ext tensorboard # Reload if loaded once
%tensorboard --logdir $config.LOG_DIR

## Start **testing** progress


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# [Param] Choose specific model weight
# MODEL_PATH = os.path.join(config.LOG_DIR, "./oralcancer20220624T1332/mask_rcnn_OralCancer_0036.h5")
MODEL_PATH = "" # Default, choosed by the last

# Create output directory
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    IMAGES_PER_GPU = 1
config = InferenceConfig()
config.display()

def get_ax(rows=1, cols=1, size=8):
    # Return a Matplotlib Axes array to be used in
    # all visualizations in the notebook.
    # Provide a central point to control graph sizes.
    # Adjust the size attribute to control how big to render images.
    
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Load test dataset
dataset_test = OralDataset()
dataset_test.load_dataset(config.DATASET_DIR, "test")
dataset_test.prepare()

# Load testing model weights
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=config.LOG_DIR)

if MODEL_PATH != "":
    weights_path = MODEL_PATH
else:
    weights_path = model.find_last()

print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

# Initial gt and prediction lists
gt_tot_per_image = np.array([])
pred_tot_per_image = np.array([])

# Initial mAP list
mAP_ = []
mprecision_ = []
mrecall_ = []
mdice = []
class_names = []
class_names_index = []
APs = []
count1 = 0

## Load and inference on testing dataset

In [ ]:
for image_id in dataset_test.image_ids:
    # Load ground truth of testing samples
    image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset_test, config, image_id)
    info = dataset_test.image_info[image_id]
    
    # Show image ID and gt info
    print("\nImage ID: {}\n".format(info["id"]))
    log("gt_class_id", gt_class_id)
    log("gt_bbox", gt_bbox)
    log("gt_mask", gt_mask)
    
    # Inference on testing sample
    molded_image = np.expand_dims(image, 0)
    results = model.detect(molded_image, verbose=0)

    # Get results
    ax = get_ax()
    r = results[0]
    
    # Concatenate all materials for confusion matrix
    if count1 == 0:
        # Concatenate gt and predicted class ID, bbox, mask for first time
        concat_gt_class_id, concat_gt_bbox, concat_gt_mask = gt_class_id, gt_bbox, gt_mask
        concat_r_rois, concat_r_masks, concat_r_class_ids, concat_r_scores = r['rois'], r['masks'], r['class_ids'], r['scores']
    else:
        # Concatenate gt class, bbox, mask
        concat_gt_class_id = np.concatenate((concat_gt_class_id, gt_class_id), axis=0)
        concat_gt_bbox = np.concatenate((concat_gt_bbox, gt_bbox), axis=0)
        concat_gt_mask = np.concatenate((concat_gt_mask, gt_mask), axis=2)
        # Concatenate predicted class, bbox, mask
        concat_r_rois = np.concatenate((concat_r_rois, r['rois']), axis=0)
        concat_r_masks = np.concatenate((concat_r_masks, r['masks']), axis=2)
        concat_r_class_ids = np.concatenate((concat_r_class_ids, r['class_ids']), axis=0)
        concat_r_scores = np.concatenate((concat_r_scores, r['scores']), axis=0)
    count1 += 1
    
    # Accumulate total gt and total prediction per image    
    if len(gt_class_id) == 0 and len(r['class_ids']) > 0:
        gt_tot_per_image = np.append(gt_tot_per_image, np.array(0))
        pred_tot_per_image = np.append(pred_tot_per_image, np.max(r['class_ids']))
        cls = np.max(r['class_ids'])
        if dataset_test.class_names[int(cls)] not in class_names:
            class_names.append(dataset_test.class_names[int(cls)])
            class_names_index.append(int(cls))            
    elif len(gt_class_id) > 0 and len(r['class_ids']) == 0:
        gt_tot_per_image = np.append(gt_tot_per_image, np.max(gt_class_id))
        pred_tot_per_image = np.append(pred_tot_per_image, np.array(0))
        cls = np.max(gt_class_id)
        if dataset_test.class_names[int(cls)] not in class_names:
            class_names.append(dataset_test.class_names[0])
            class_names_index.append(0)            
    elif len(r['class_ids']) == 0 and len(gt_class_id) == 0:
        gt_tot_per_image = np.append(gt_tot_per_image, np.array(0))
        pred_tot_per_image = np.append(pred_tot_per_image, np.array(0))
        if dataset_test.class_names[0] not in class_names:
            class_names.append(dataset_test.class_names[0])
            class_names_index.append(0)            
    else:
        gt_tot_per_image = np.append(gt_tot_per_image, np.max(gt_class_id))
        pred_tot_per_image = np.append(pred_tot_per_image, np.max(r['class_ids']))
        cls = np.max(gt_class_id)
        if dataset_test.class_names[int(cls)] not in class_names:
            class_names.append(dataset_test.class_names[int(cls)])
            class_names_index.append(int(cls))
    
    # Calculate dice score of each class
    dice = []
    cls_index = []
    
    for m, cls_name in enumerate(r['class_ids']):
        cls_index.append(cls_name)

    for j in range(len(gt_class_id)):
        gt = gt_class_id[j].astype(np.int32) * np.array(gt_mask.astype(np.int32)[:,:,j])
        for k in range(len(cls_index)):
            seg = cls_index[k].astype(np.int32) * r["masks"].astype(np.int32)[:,:,k]
            score = np.sum(seg[gt==gt_class_id[j].astype(np.int32)])*2.0 / (np.sum(seg) + np.sum(gt))
            if score <= 1:
                dice.append(score)
    
    if np.mean(dice) <= 1:
        print('\nMean dice similarity score is {}'.format(np.mean(dice)))
        mdice.append(np.mean(dice))
    
    # Calculate average precision
    AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'])
    APs.append(AP)
    
    # Output predicted mask results
    image_display = visualize.display_instances(config.OUTPUT_DIR, info["id"], image, r['rois'], r['masks'], r['class_ids'], dataset_test.class_names, r['scores'], ax=ax)
    plt.imshow(image_display)
    plt.savefig("{}/{}".format(config.OUTPUT_DIR, str(info["id"])))
    plt.show()
    plt.close()

## Plot evaluation figures

In [ ]:
import seaborn as sn
from pandas import DataFrame
from sklearn.metrics import confusion_matrix

# Print mAP and mean dice scores
print("mAP @ IoU=50: ", np.mean(APs))
print("mean Dice score: ", np.mean(mdice))

# (1) Plot confusion matrix
print('\n[Fig] Confusion matrix')

y_test = np.array(gt_tot_per_image)
predictions = np.array(pred_tot_per_image)

confm = confusion_matrix(y_test, predictions)
df_cm = DataFrame(confm, index=[dataset_test.class_names], columns=[dataset_test.class_names])

sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')

plt.title('Confusion matrix (per image)')
plt.savefig(config.OUTPUT_DIR+"/Confusion_matrix.png")
plt.show()
plt.close()

# (2) Calculate F1 scores & plot Precision-Recall curve 
print('\n[Fig] Precision-Recall curve')

fig, ax = plt.subplots(1, 1, figsize=(9, 6), tight_layout=True)

# Customized color list
colorlist = ['Green', 'Yellow', 'Red']
F1_scores = []

for class_id in range(1, len(dataset_test.class_names)):
    mAP, precisions, recalls, _ =\
            utils.compute_ap_pre_class(concat_gt_bbox, concat_gt_class_id, concat_gt_mask, concat_r_rois, concat_r_class_ids, concat_r_scores, concat_r_masks, class_id)
    mAR, _ = utils.compute_recall(concat_r_rois, concat_gt_bbox, iou=0.5) 
    
    F1_scores = (2* (np.mean(precisions) * np.mean(recalls)))/(np.mean(precisions) + np.mean(recalls))
    ax.plot(recalls, precisions, color = colorlist[class_id], label=dataset_test.class_names[class_id] + ' mAP=%.2f, F1 score=%.2f' % (mAP, F1_scores))

ax.set_xlabel('Recall (False Positive Rate)')
ax.set_ylabel('Precision (True Positive Rate)')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.legend(loc="upper right", prop={'size': 12})

mmAP, mprecisions, mrecalls, overlaps =\
            utils.compute_ap(concat_gt_bbox, concat_gt_class_id, concat_gt_mask,
                     concat_r_rois, concat_r_class_ids, concat_r_scores, concat_r_masks)
mmAR, _ = utils.compute_recall(concat_r_rois, concat_gt_bbox, iou=0.5) 
F1_scores = (2* (np.mean(mprecisions) * np.mean(mrecalls)))/(np.mean(mprecisions) + np.mean(mrecalls))
ax.plot(mrecalls, mprecisions, color = 'Blue', label='Total mAP=%.2f, F1 score=%.2f, Mean dice similarity score=%.2f' % (mmAP, F1_scores, np.mean(mdice)))

plt.legend()
plt.title('Precision-Recall curve')
plt.savefig(config.OUTPUT_DIR+"/PR_curve.png")
plt.show()
plt.close()